In [1]:
import pandas as pd

In [7]:
# Ler os dados
w_data = pd.read_csv("../Data/w_data.csv", sep=';')
p_data = pd.read_csv("../Data/p_data.csv", sep=';')

In [5]:
w_data.head()

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target
0,State-gov,77516.0,2174.0,0.0,40.0,49084,<=50K
1,Self-emp-not-inc,83311.0,0.0,0.0,13.0,68553,<=50K
2,Private,215646.0,0.0,0.0,40.0,143654,<=50K
3,Private,234721.0,0.0,0.0,40.0,41510,<=50K
4,Private,338409.0,0.0,0.0,40.0,195025,<=50K


In [8]:
p_data.head()

,Age,Sex,ID,Race,Country,Education,Education-Num,Marital Status,Relationship
0,39.0,Male,49084,White,United-States,Bachelors,13.0,Never-married,Not-in-family
1,50.0,Male,68553,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
2,38.0,Male,143654,White,United-States,HS-grad,9.0,Divorced,Not-in-family
3,53.0,Male,41510,Black,United-States,11th,7.0,Married-civ-spouse,Husband
4,28.0,Female,195025,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife


In [26]:
# Fundir os dados a partir da variável ID
merged_data = pd.merge(w_data, p_data, on='ID', how='outer')
merged_data

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship
0,State-gov,77516.0,2174.0,0.0,40.0,49084,<=50K,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family
1,Self-emp-not-inc,83311.0,0.0,0.0,13.0,68553,<=50K,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
2,Private,215646.0,0.0,0.0,40.0,143654,<=50K,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family
3,Private,234721.0,0.0,0.0,40.0,41510,<=50K,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband
4,Private,338409.0,0.0,0.0,40.0,195025,<=50K,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48838,Private,215419.0,0.0,0.0,36.0,735617153,<=50K.,39.0,Female,White,United-States,Bachelors,13.0,Divorced,Not-in-family
48839,?,321403.0,0.0,0.0,40.0,1131807446,<=50K.,64.0,Male,Black,United-States,HS-grad,9.0,Widowed,Other-relative
48840,Private,374983.0,0.0,0.0,50.0,781422087,<=50K.,38.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
48841,Private,83891.0,5455.0,0.0,40.0,161771296,<=50K.,44.0,Male,Asian-Pac-Islander,United-States,Bachelors,13.0,Divorced,Own-child


In [27]:
# Quantos ID têm em comum?
total_ids_w = w_data['ID'].unique()
total_ids_p = p_data['ID'].unique()
ids_not_common = []
ids_common = []
for id_p in total_ids_p:
    if id_p not in total_ids_w:
        ids_not_common.append(id_p)
    else:
        ids_common.append(id_p)
        
for id_w in total_ids_w:
    if id_w not in total_ids_p:
        ids_not_common.append(id_w)
    else:
        ids_common.append(id_w)
        
ids_common = set(ids_common)

print("Numero de IDs comuns:", len(ids_common))
print("IDs que não têm em comum:", ids_not_common)

Numero de IDs comuns: 48842
IDs que não têm em comum: [1344995623]


In [32]:
# Corrigir os Target a apenas dois valores distintos ‘<=50k’ e ‘>50k’
print(f"Antes: {merged_data['Target'].unique()}")
merged_data['Target'] = merged_data['Target'].str.replace(" ", "")
merged_data['Target'] = merged_data['Target'].str.replace(".", "")
merged_data['Target'] = merged_data['Target'].str.replace("K", "k")
print(f"Depois: {merged_data['Target'].unique()}")

Antes: ['<=50K' '>50K' nan]
Depois: ['<=50k' '>50k' nan]


In [43]:
# Calcular a proporção entre pessoas com salários >50k em relação ao total com pessoas que difiram em sexo, raça e nível educacional
grouped_data = merged_data.groupby(['Sex', 'Race', 'Education'])
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")

   Pct   |  Total  |  Grupo
  0.00%  |     10  |  (' Female', ' Amer-Indian-Eskimo', ' 10th')
  0.00%  |      7  |  (' Female', ' Amer-Indian-Eskimo', ' 11th')
  0.00%  |      2  |  (' Female', ' Amer-Indian-Eskimo', ' 1st-4th')
  0.00%  |      3  |  (' Female', ' Amer-Indian-Eskimo', ' 7th-8th')
  0.00%  |      3  |  (' Female', ' Amer-Indian-Eskimo', ' 9th')
 12.50%  |      8  |  (' Female', ' Amer-Indian-Eskimo', ' Assoc-acdm')
 11.11%  |     18  |  (' Female', ' Amer-Indian-Eskimo', ' Assoc-voc')
 25.00%  |     12  |  (' Female', ' Amer-Indian-Eskimo', ' Bachelors')
 50.00%  |      2  |  (' Female', ' Amer-Indian-Eskimo', ' Doctorate')
  3.39%  |     59  |  (' Female', ' Amer-Indian-Eskimo', ' HS-grad')
 33.33%  |      6  |  (' Female', ' Amer-Indian-Eskimo', ' Masters')
100.00%  |      1  |  (' Female', ' Amer-Indian-Eskimo', ' Prof-school')
  5.56%  |     54  |  (' Female', ' Amer-Indian-Eskimo', ' Some-college')
 14.29%  |      7  |  (' Female', ' Asian-Pac-Islander', ' 10th')
 

In [48]:
# Calcular a proporção entre pessoas com salários >50k em relação ao total com pessoas que difiram em sexo, raça e nível educacional
grouped_data = merged_data.groupby(['Sex', 'Race'])
print('-'*50)
print(f'{"Sex / Race":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")
        
grouped_data = merged_data.groupby(['Sex', 'Education'])
print('-'*50)
print(f'{"Sex / Education":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")
        
grouped_data = merged_data.groupby(['Race', 'Education'])
print('-'*50)
print(f'{"Race / Education":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")

--------------------------------------------------
                    Sex / Race                    
   Pct   |  Total  |  Grupo
  8.11%  |    185  |  (' Female', ' Amer-Indian-Eskimo')
 13.35%  |    517  |  (' Female', ' Asian-Pac-Islander')
  5.72%  |   2308  |  (' Female', ' Black')
  7.10%  |    155  |  (' Female', ' Other')
 11.84%  |  13027  |  (' Female', ' White')
 14.04%  |    285  |  (' Male', ' Amer-Indian-Eskimo')
 33.93%  |   1002  |  (' Male', ' Asian-Pac-Islander')
 18.26%  |   2377  |  (' Male', ' Black')
 15.54%  |    251  |  (' Male', ' Other')
 31.54%  |  28734  |  (' Male', ' White')
--------------------------------------------------
                 Sex / Education                  
   Pct   |  Total  |  Grupo
  0.88%  |    457  |  (' Female', ' 10th')
  1.69%  |    650  |  (' Female', ' 11th')
  1.90%  |    211  |  (' Female', ' 12th')
  0.00%  |     61  |  (' Female', ' 1st-4th')
  3.15%  |    127  |  (' Female', ' 5th-6th')
  0.42%  |    239  |  (' Female', ' 7

In [49]:
# Calcular a proporção entre pessoas com salários >50k em relação ao total com pessoas que difiram em sexo, raça e nível educacional
grouped_data = merged_data.groupby('Sex')
print('-'*50)
print(f'{"Sex":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")
        
grouped_data = merged_data.groupby('Education')
print('-'*50)
print(f'{"Education":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")
        
grouped_data = merged_data.groupby('Race')
print('-'*50)
print(f'{"Race":^50s}')
print("   Pct   |  Total  |  Grupo")
for name, group in grouped_data:
    total_count = len(group)
    high_income_count = len(group[group['Target'] == '>50k'])
    if total_count > 0:
        proportion = high_income_count / total_count
        print(f"{proportion:7.2%}  |  {total_count:5d}  |  {name}")

--------------------------------------------------
                       Sex                        
   Pct   |  Total  |  Grupo
 10.93%  |  16192  |   Female
 30.37%  |  32649  |   Male
--------------------------------------------------
                    Education                     
   Pct   |  Total  |  Grupo
  6.26%  |   1389  |   10th
  5.08%  |   1812  |   11th
  7.31%  |    657  |   12th
  3.24%  |    247  |   1st-4th
  5.30%  |    509  |   5th-6th
  6.49%  |    955  |   7th-8th
  5.42%  |    756  |   9th
 25.80%  |   1601  |   Assoc-acdm
 25.33%  |   2061  |   Assoc-voc
 41.28%  |   8024  |   Bachelors
 72.56%  |    594  |   Doctorate
 15.86%  |  15784  |   HS-grad
 54.91%  |   2657  |   Masters
  1.20%  |     83  |   Preschool
 73.98%  |    834  |   Prof-school
 18.96%  |  10878  |   Some-college
--------------------------------------------------
                       Race                       
   Pct   |  Total  |  Grupo
 11.70%  |    470  |   Amer-Indian-Eskimo
 26.93%